<a href="https://colab.research.google.com/github/vertumno/probabilidade-analise/blob/master/Discovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [17]:
turmas_url = 'http://bit.do/turmas_ufrn_2015_2'
turmas = pd.read_csv(turmas_url, sep=';')
turmas.head()

,id_turma,codigo_turma,id_docente_interno,id_docente_externo,observacao,id_componente_curricular,nivel_ensino,campus_turma,local,ano,...,tipo,distancia,data_consolidacao,agrupadora,id_turma_agrupadora,qtd_aulas_lancadas,situacao_turma,convenio,modalidade_participantes,Unnamed: 25
0,57560974,01,5755580.0,NaN,NaN,55731,LATO SENSU,NaN,Caicó,2015,...,REGULAR,f,2016-02-29 16:58:56.653,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN
1,57560975,01,5756402.0,NaN,NaN,55770,LATO SENSU,NaN,Caicó,2015,...,REGULAR,f,2016-09-26 00:28:07.764,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN
2,57560976,01,22193.0,NaN,NaN,55743,LATO SENSU,NaN,Caicó,2015,...,REGULAR,f,2016-01-07 16:54:55.815,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN
3,57560977,01,5755386.0,NaN,NaN,55734,LATO SENSU,NaN,Caicó,2015,...,REGULAR,f,2016-01-29 17:20:44.777,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN
4,57560982,01,5755973.0,NaN,NaN,55735,LATO SENSU,NaN,Currais Novos,2015,...,REGULAR,f,2016-01-27 16:44:08.695,f,NaN,30.0,CONSOLIDADA,NaN,Presencial,NaN


In [19]:
rptmi_turma_1 = turmas[turmas['id_componente_curricular'] == 55031]
rptmi_turma_1.head()

,id_turma,codigo_turma,id_docente_interno,id_docente_externo,observacao,id_componente_curricular,nivel_ensino,campus_turma,local,ano,...,tipo,distancia,data_consolidacao,agrupadora,id_turma_agrupadora,qtd_aulas_lancadas,situacao_turma,convenio,modalidade_participantes,Unnamed: 25
3461,57565214,01,5756968.0,NaN,NaN,55031,GRADUAÇÃO,Campus Central,3H2 - Setor III,2015,...,REGULAR,f,2015-12-19 10:19:45.665,f,NaN,216.0,CONSOLIDADA,NaN,Presencial,NaN
3462,57565214,01,5756386.0,NaN,NaN,55031,GRADUAÇÃO,Campus Central,3H2 - Setor III,2015,...,REGULAR,f,2015-12-19 10:19:45.665,f,NaN,216.0,CONSOLIDADA,NaN,Presencial,NaN
3471,57565222,02,5757585.0,NaN,NaN,55031,GRADUAÇÃO,Campus Central,3H2 - Setor III,2015,...,REGULAR,f,2015-12-18 23:54:59.423,f,NaN,222.0,CONSOLIDADA,NaN,Presencial,NaN
3472,57565222,02,5755819.0,NaN,NaN,55031,GRADUAÇÃO,Campus Central,3H2 - Setor III,2015,...,REGULAR,f,2015-12-18 23:54:59.423,f,NaN,222.0,CONSOLIDADA,NaN,Presencial,NaN
3474,57565224,03,5757669.0,NaN,NaN,55031,GRADUAÇÃO,Campus Central,3H2 - Setor III,2015,...,REGULAR,f,2015-12-18 12:29:06.215,f,NaN,220.0,CONSOLIDADA,NaN,Presencial,NaN
